In [3]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics import *
import random
import matplotlib.pyplot as plt

In [2]:
print(os.getcwd())

/root/TSTF/main


* 데이터 파일
* 예측 결과

논문에 있는 거랑 정확히 같지는 않은 것 같은데, 결과도 전부 주어지지 않았고.

In [ ]:
data = ['coin']
model_name = 'trTFTF'
num = 2  # Model k per loss

for d in range(len(data)):
    #############################################################################################
    target_X = pd.read_csv(f"../data/{data[d]}/train_input_7.csv").iloc[:, 1:].values.astype(np.float32)
    target_y = pd.read_csv(f"../data/{data[d]}/train_output_7.csv").iloc[:, 1:].values.astype(np.float32)

    target_X_val = target_X[-round(target_X.shape[0] * 0.2):, :].astype(np.float32)
    target_y_val = target_y[-round(target_y.shape[0] * 0.2):].astype(np.float32)

    target_X = target_X[:-round(target_X.shape[0] * 0.2), :].astype(np.float32)
    target_y = target_y[:-round(target_y.shape[0] * 0.2)].astype(np.float32)
    test_X = pd.read_csv(f"../data/{data[d]}/val_input_7.csv").iloc[:, 1:].values.astype(np.float32)
    test_y = pd.read_csv(f"../data/{data[d]}/val_output_7.csv").iloc[:, 1:].values.astype(np.float32)

    #############################################################################################
    folder_path = 'resulttf/test/'
    folder_path2 = 'resulttf/val/'

    all_files = os.listdir(folder_path)
    all_files2 = os.listdir(folder_path2)
    print(f"--- 파일 검색 디버깅 ---")
    print(f"검색 중인 폴더: {folder_path}")
    print(f"폴더 안의 모든 파일: {all_files}")
    search_prefix = f"{model_name}_{data[d]}"
    print(f"찾고 있는 파일 시작 부분(prefix): '{search_prefix}'")
    print("--- 디버깅 끝 ---")
    files = sorted([f for f in all_files if f.startswith(f"{model_name}_{data[d]}") and f.endswith("pred.csv")])
    dataframes = [pd.read_csv(os.path.join(folder_path, f)) for f in files]

    files2 = sorted([f for f in all_files2 if f.startswith(f"{model_name}_{data[d]}") and f.endswith("pred.csv")])
    dataframes2 = [pd.read_csv(os.path.join(folder_path2, f)) for f in files2]

    ## all의 차원은 (5, 100, 528, 24)가 됨 (test)
    ## all_val의 차원은 (5, 100, 145, 24)가 됨 (validation)
    all = np.array([np.stack(dataframes[i].iloc[:, 1:].values.reshape(num, -1, target_y.shape[1])) for i in range(len(dataframes))])
    all_val = np.array([np.stack(dataframes2[i].iloc[:, 1:].values.reshape(num, -1, target_y.shape[1])) for i in range(len(dataframes2))])
    #############################################################################################
    all_ens_lst, all_ens_rmse_lst = [], []
    bolt_lst, bolt_rmse_lst = [], []

    inv_lst, inv_rmse_lst = [], []  ### NEW: 역수 가중치 결과 저장용
    mean_rmse_lst = []  ### NEW: mean 평가 지표 기록용

    p = 2
    b = 50
    np.random.seed(100)  ### NEW: mean 평가 지표 기록용

    for i in range(b):
        # --------------------------------------------------------------
        # 1) 무작위 인덱스 샘플링
        # matrix = np.arange(100).reshape(10, 10)
        nums1 = random.sample(range(num), k=p)
        nums2 = random.sample(range(num), k=p)
        nums3 = random.sample(range(num), k=p)
        nums4 = random.sample(range(num), k=p)
        nums5 = random.sample(range(num), k=p)

        # nums1 = matrix[i]
        # nums2 = matrix[i]
        # nums3 = matrix[i]
        # nums4 = matrix[i]
        # nums5 = matrix[i]
        # --------------------------------------------------------------
        # 2) all_Ens (median 앙상블은 그대로)
        score = np.concatenate([all[0][nums1], all[1][nums2],
                                all[2][nums3], all[3][nums4],
                                all[4][nums5]], axis=0)
        ## 각 손실함수 별 10개씩 추출

        all_ens = np.median(score, axis=0).flatten()
        all_ens_rmse = np.sqrt(mean_squared_error(test_y.flatten(), all_ens))
        all_ens_lst.append(all_ens)
        all_ens_rmse_lst.append(all_ens_rmse)

        # --------------------------------------------------------------

        # --------------------------------------------------------------
        # 4) mean 평가지표(다섯 RMSE의 평균) 기록 ### NEW
        mean_rmse = np.mean(score, axis=0).flatten()
        all_mean_rmse = np.sqrt(mean_squared_error(test_y.flatten(), mean_rmse))
        mean_rmse_lst.append(all_mean_rmse)
        ####################################################################
        # Boltzmann Weights Ensemble
        mae_val = np.median(all_val[0][nums1], axis=0).flatten()  # .shape
        mape_val = np.median(all_val[1][nums2], axis=0).flatten()
        mase_val = np.median(all_val[2][nums3], axis=0).flatten()
        mse_val = np.median(all_val[3][nums4], axis=0).flatten()
        smape_val = np.median(all_val[4][nums5], axis=0).flatten()

        rmse_mae_val = np.sqrt(mean_squared_error(target_y_val.flatten(), mae_val))
        rmse_mape_val = np.sqrt(mean_squared_error(target_y_val.flatten(), mape_val))
        rmse_mase_val = np.sqrt(mean_squared_error(target_y_val.flatten(), mase_val))
        rmse_mse_val = np.sqrt(mean_squared_error(target_y_val.flatten(), mse_val))
        rmse_smape_val = np.sqrt(mean_squared_error(target_y_val.flatten(), smape_val))

        fin_pred_mae = np.median(all[0][nums1], axis=0).flatten()
        fin_pred_mape = np.median(all[1][nums2], axis=0).flatten()
        fin_pred_mase = np.median(all[2][nums3], axis=0).flatten()
        fin_pred_mse = np.median(all[3][nums4], axis=0).flatten()
        fin_pred_smape = np.median(all[4][nums5], axis=0).flatten()

        performance = np.array([rmse_mae_val, rmse_mape_val, rmse_mase_val, rmse_mse_val, rmse_smape_val])

        bolt_rmse_lst_ = []
        bolt = []
        for beta in np.arange(1, 10.2, 0.2):
            # for beta in np.arange(1, 2):
            # for beta in np.arange(3, 4):
            # for beta in np.arange(5, 6):
            # beta = 1 # 조정 파라미터
            weights = np.exp(-beta * performance)
            gd = np.concatenate([fin_pred_mae.flatten().reshape(1, -1),
                                 fin_pred_mape.flatten().reshape(1, -1),
                                 fin_pred_mase.flatten().reshape(1, -1),
                                 fin_pred_mse.flatten().reshape(1, -1),
                                 fin_pred_smape.flatten().reshape(1, -1)], axis=0)

            normalized_weights = weights / np.sum(weights)

            # 각 모델의 예측값에 가중치를 부여하여 앙상블 예측 생성
            ensemble_prediction = np.dot(normalized_weights, gd)
            bolt.append(ensemble_prediction)
            bolt_rmse = np.sqrt(mean_squared_error(test_y.flatten(), ensemble_prediction.flatten()))
            bolt_rmse_lst_.append(bolt_rmse)

        bolt_lst.append(bolt)
        bolt_rmse_lst.append(bolt_rmse_lst_)

        # --------------------------------------------------------------
        # 7) **역수 가중치 앙상블** ### NEW
        inv_weights = 1.0 / performance
        inv_weights /= np.sum(inv_weights)  # 정규화

        ens_pred_inv = np.dot(inv_weights, gd)
        inv_lst.append(ens_pred_inv)

        inv_rmse = np.sqrt(mean_squared_error(test_y.flatten(),
                                              ens_pred_inv.flatten()))
        inv_rmse_lst.append(inv_rmse)

    a = np.min(np.array(bolt_rmse_lst), axis=0)

    print(np.argmin(a) * 0.2)
    summary = pd.DataFrame({'BOLT': np.array(bolt_rmse_lst).T[np.argmin(a)],
                            'Median': all_ens_rmse_lst,
                            'INV': inv_rmse_lst,
                            'Mean': mean_rmse_lst})
    os.makedirs("inference", exist_ok = True)
    summary.to_csv(f'inference/{model_name}_{data[d]}_summary.csv')
    summary.describe()

--- 파일 검색 디버깅 ---
검색 중인 폴더: resulttf/test/
폴더 안의 모든 파일: ['trTFTF_coin_mape_pred.csv', 'trTFTF_coin_mse_pred.csv', 'trTFTF_coin_smape_pred.csv', 'trTFTF_coin_mae_pred.csv', 'trTFTF_coin_mase_pred.csv']
찾고 있는 파일 시작 부분(prefix): 'trTFTF_coin'
--- 디버깅 끝 ---
0.0


In [32]:
data = ['coin']
rec = []
for i in range(len(data)):
    d= pd.read_csv(f'inference/{model_name}_{data[i]}_summary.csv').iloc[:,1:].describe().T.iloc[:,1:3].round(5)
    d['mean(std)'] = d.apply(
    lambda r: f"{r['mean']:.5f} ({r['std']:.5f})", axis=1)
    
    rec.append(d.loc[:,'mean(std)'])

dfff = pd.DataFrame(rec).T

dfff.columns = data
dfff.to_csv(f'{model_name}_result_ensemble.csv')

In [33]:
dfff = pd.DataFrame(rec).T

dfff.columns = data
dfff.to_csv(f'{model_name}_result_ensemble.csv')

In [7]:
dfff

,coin
BOLT,23.75481 (0.00000)
Median,33.10408 (0.00000)
INV,29.65532 (0.00000)
Mean,37.08300 (0.00000)


In [8]:
a=np.min(np.array(bolt_rmse_lst),axis=0)

print(np.argmin(a)*0.2)
summary= pd.DataFrame({'BOLT' : np.array(bolt_rmse_lst).T[np.argmin(a)],
             'Median' : all_ens_rmse_lst,
             'INV' :inv_rmse_lst,
            'Mean': mean_rmse_lst})
summary.to_csv(f'inference/{model_name}_{data}_summary.csv')
summary.describe()

0.0


,BOLT,Median,INV,Mean
count,5.000000e+01,5.000000e+01,50.00000,50.000000
mean,2.375481e+01,3.310408e+01,29.65532,37.082997
std,1.076635e-14,2.153270e-14,0.00000,0.000000
min,2.375481e+01,3.310408e+01,29.65532,37.082997
25%,2.375481e+01,3.310408e+01,29.65532,37.082997
50%,2.375481e+01,3.310408e+01,29.65532,37.082997
75%,2.375481e+01,3.310408e+01,29.65532,37.082997
max,2.375481e+01,3.310408e+01,29.65532,37.082997


In [9]:
summary.iloc[0]

BOLT      23.754809
Median    33.104083
INV       29.655320
Mean      37.082997
Name: 0, dtype: float64